In [1]:
import os
from collections import defaultdict
import re
import gensim
import numpy as np

pat = re.compile(r"[\w]+|[.,!?;|]")
ko_model = gensim.models.fasttext.load_facebook_model('cc.ko.300.bin.gz') # https://fasttext.cc/docs/en/crawl-vectors.html 에서 다운받고, 같은 폴더에 넣기

# file list 불러오기
os.chdir('PDFtoTEXT')
total_list = dict()
for cat in os.listdir():
    total_list[cat] = [(i, i.split('_')[1].split('(')[0]) for i in os.listdir(cat)]
        
# 각 파일을 열어서, 안의 내용 저장 (stopwords 처리 및 tokenizing)
total = defaultdict(lambda: defaultdict(lambda : []))
for k, v in total_list.items():
    for i, j in v:
        with open(os.path.join(k, i), 'r', encoding='utf-8') as f:
            line = f.readlines()
        total[k][j].append(pat.findall(re.sub('[-_0-9.,]', ' ', ''.join(line))))
        
# 단어 목록 생성
vocab_candidate = []
for k in total.keys():
    for k_ in total[k].keys():
        for v_ in total[k][k_]:
            vocab_candidate.extend(v_)
vocab_candidate = list(set(vocab_candidate))

# 단어 목록 중 pre-trained embedding matrix에 있는 단어 추출 및 embedding matrix 추출
vocab = [i for i in vocab_candidate if ko_model.wv.vocab.get(i)]
vocab_size = len(vocab)

embedding_matrix = np.zeros((vocab_size + 1, 300))

word2idx = dict()
for i, v in enumerate(vocab):
    idx = ko_model.wv.index2word.index(v)
    word2idx[v] = i + 1
    embedding_matrix[i + 1] = ko_model.wv.vectors[idx]
    
# OOV words 처리
for k in total.keys():
    for k_ in total[k].keys():
        for e, v_ in enumerate(total[k][k_]):
            total[k][k_][e] = [word2idx[j] for j in v_ if word2idx.get(j)]